In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')
print(train.size)
train.replace([np.inf, -np.inf], np.nan, inplace=True)
train.dropna(inplace=True)
print(train.size)
# take a sample of the training data
train = train.sample(frac=0.01)
print(train.size)

In [ ]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# Helper function for adding all the engineered features
def feature_engineering(df):
    df['Upper_Shadow'] = upper_shadow(df)
    df['Lower_Shadow'] = lower_shadow(df)
    return df

train = feature_engineering(train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train[['Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Upper_Shadow', 'Lower_Shadow']].values, train['Target'].values)

In [ ]:
# Start to predict
import math

import gresearch_crypto
env = gresearch_crypto.make_env()   # initialize the environment

def predict(row):
    row = feature_engineering(row)
    p_val = model.predict([row[['Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Upper_Shadow', 'Lower_Shadow']].values])[0]
    return p_val

iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['Target'] = test_df.apply(lambda row: predict(row), axis = 1)
    #print(sample_prediction_df)
    #sample_prediction_df['Target'] = 0  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
    